In [ ]:
#IMPORTAR LIBRERÍAS
#Advertencias
import warnings
warnings.filterwarnings('ignore')

#Tratamiento de datos
import numpy as np
import pandas as pd

#Gráficos
import matplotlib.pyplot as plt
import seaborn as sb
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

#Modelado y pronóstico
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

#Evaluación
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
#Configuración de gráficos
plt.style.use('fivethirtyeight')

In [ ]:
#Importar datos
#Datos:
#Time: fecha y hora del registro del consumo
#Date: fecha del registro del dato
#Demand: valor de la demana de energía en Md
#Temperatura: temperatura del sitio
#Holiday: indica si el día es o no festivo (incluye vacaciones)

url = ('https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/data/vic_elec.csv')
datos = pd.read_csv(url)
datos.head()

In [ ]:
datos.info()

In [ ]:
#Transformación de la variable Time a fecha-hora
datos['Time'] = pd.to_datetime(datos['Time'], format = '%Y-%m-%dT%H:%M:%SZ')
datos = datos.set_index('Time')
datos = datos.asfreq('30min')
datos = datos.sort_index()
datos.head(20)

In [ ]:
datos.info()

In [ ]:
#Validar que el índice temporal está completo
(datos.index == pd.date_range(start = datos.index.min(),
                            end = datos.index.max(),
                            freq = datos.index.freq)).all()

In [ ]:
#datos = datos.drop('Date', axis = 1)
datos = datos.resample(rule = 'H', closed = 'left', label = 'right').mean()
datos.head()

In [ ]:
#Construcción de conjuntos de entrenamiento, prueba y validación
datos = datos.loc['2012-01-01 00:00:00' : '2014-12-30 23:00:00']
finEntrenamiento = '2013-12-31 23:59:00'
finPrueba = '2014-11-30 23:59:00'

datosEntrenamiento = datos.loc[:finEntrenamiento, :]
datosPrueba = datos.loc[finEntrenamiento:finPrueba, :]
datosValidacion = datos.loc[finPrueba:, :]

print(f'Fechas de entrenamiento: {datosEntrenamiento.index.min()} --- {datosEntrenamiento.index.max()} (n = {len(datosEntrenamiento)})')
print(f'Fechas de prueba: {datosPrueba.index.min()} --- {datosPrueba.index.max()} (n = {len(datosPrueba)})')
print(f'Fechas de validación: {datosValidacion.index.min()} --- {datosValidacion.index.max()} (n = {len(datosValidacion)})')


In [ ]:
#Gráfico de serie temporal
fig, ax = plt.subplots(figsize = (12, 4))
datosEntrenamiento.Demand.plot(ax = ax, label = 'Entrenamiento')
datosPrueba.Demand.plot(ax = ax, label = 'Prueba')
datosValidacion.Demand.plot(ax = ax, label = 'Validación')
ax.set_title('Demanda eléctrica')
ax.legend()
plt.show()

In [ ]:
#Identificarla estacionalidad por mes
fig, ax = plt.subplots(figsize =(10, 5))
datos['mes'] = datos.index.month
datos.boxplot(column = 'Demand', by = 'mes', ax = ax)
datos.groupby('mes')['Demand'].median().plot(style = 'o-', ax = ax)
ax.set_ylabel('Demanda')
ax.set_title('Distribución de demanda por mes')
fig.suptitle('')
plt.show()

In [ ]:
#Identificarla estacionalidad por día semana
fig, ax = plt.subplots(figsize =(10, 5))
datos['dia_semana'] = datos.index.day_of_week + 1
datos.boxplot(column = 'Demand', by = 'dia_semana', ax = ax)
datos.groupby('dia_semana')['Demand'].median().plot(style = 'o-', ax = ax)
ax.set_ylabel('Demanda')
ax.set_title('Distribución de demanda por día de la semana')
fig.suptitle('')
plt.show()

In [ ]:
#Identificarla estacionalidad por hora del día
fig, ax = plt.subplots(figsize =(10, 5))
datos['hora_dia'] = datos.index.hour + 1
datos.boxplot(column = 'Demand', by = 'hora_dia', ax = ax)
datos.groupby('hora_dia')['Demand'].median().plot(style = 'o-', ax = ax)
ax.set_ylabel('Demanda')
ax.set_title('Distribución de demanda por hora del día')
fig.suptitle('')
plt.show()

In [ ]:
#Efecto de los días festivos
fig, ax = plt.subplots(figsize = (10, 5))
sb.violinplot(x = 'Demand',
             y = 'Holiday',
             data = datos.assign(Holiday = datos.Holiday.astype(str)),
             ax = ax)
ax.set_title('Distribución de la demanda en festivos')
ax.set_xlabel('Demanda')
ax.set_ylabel('Festivo: 0 > No festivo, 2 > Festivo')

In [ ]:
#Gráfico de Autocorrelación 
fig, ax = plt.subplots(figsize = (7, 3))
plot_acf(datos.Demand, ax = ax, lags = 100)
plt.show()

In [ ]:
#Gráfico de Autocorrelación Parcial 
fig, ax = plt.subplots(figsize = (7, 3))
plot_pacf(datos.Demand, ax = ax, lags = 100)
plt.show()

In [ ]:
#Modelado
forecaster = ForecasterAutoreg(regressor = DecisionTreeRegressor(),
                              lags = 24,
                              transformer_y = StandardScaler())
forecaster.fit(y = datos.loc[:finPrueba, 'Demand'])
forecaster

In [ ]:
metrica, predicciones = backtesting_forecaster(forecaster = forecaster,
                                              y = datos.Demand,
                                              initial_train_size = len(datos.loc[:finPrueba]),
                                              steps = 24,
                                              metric = 'mean_absolute_error',
                                              refit = False,
                                              verbose = True)

In [ ]:
fig, ax = plt.subplots(figsize = (12, 5))
datos.loc[predicciones.index, 'Demand'].plot(ax = ax, label = 'Prueba')
predicciones.plot(ax = ax, label = 'Pronóstico')
ax.legend()
plt.show()

In [ ]:
metrica

In [ ]:
#Búsqueda de hiperparámetros
forecaster = ForecasterAutoreg (regressor = DecisionTreeRegressor(),
                                 lags = 24,
                                 transformer_y = StandardScaler())

#Lags utilizados como predictores
lags_grid = [5, 24, [1, 2, 3, 23, 24, 25, 47, 48, 49]]

#Hiperparámetros del regresor
param_grid = {'max_depth': [5, 7, 9, 11], 
             'criterion':['squared_error', 'absolute_error']}

resultados_grid = grid_search_forecaster(forecaster = forecaster,
                                        y = datos.loc[:finPrueba, 'Demand'],
                                        param_grid = param_grid,
                                        lags_grid = lags_grid,
                                        steps = 24,
                                        metric = 'mean_absolute_error',
                                        refit = False,
                                        initial_train_size = len(datos[:finEntrenamiento]),
                                        fixed_train_size = False,
                                        return_best = True,
                                        verbose = False)

In [ ]:
resultados_grid

In [ ]:
forecaster

In [ ]:
metrica, predicciones = backtesting_forecaster(forecaster = forecaster,
                                              y = datos.Demand,
                                              initial_train_size = len(datos.loc[:finPrueba]),
                                              steps = 24,
                                              metric = 'mean_absolute_error',
                                              refit = False,
                                              verbose = False)

In [ ]:
fig, ax = plt.subplots(figsize = (12, 5))
datos.loc[predicciones.index, 'Demand'].plot(ax = ax, label = 'Prueba')
predicciones.plot(ax = ax, label = 'Pronóstico')
ax.legend()
plt.show()

In [ ]:
metrica

In [ ]:
forecaster.get_feature_importance()